# DMX Dev Notebook
Use this notebook to develop code to automatic DMX light fixtures to system sound

In [117]:
import sounddevice as sd
import numpy as np
import time
from tqdm import tqdm

In [118]:
from PyDMXControl.controllers import OpenDMXController
from PyDMXControl.profiles.Generic import Dimmer

In [119]:

# Create an instance of the uDMX controller, 
#  this holds all the fixture information and outputs it.
# This will start outputting data immediately.
dmx = OpenDMXController()

/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 0.03426106770833215ms, took 16.700927734375ms
  warn("Ticker loop behind by {:,}ms, took {:,}ms".format(-wait_dur, loop_dur))


In [121]:
# create a dictionary of fixtures
# channels 1-4 are dimmers
# channel 5 is strobe intensity, 6 is strobe speed
lights = {}
for i in range(1, 5):
    lights[str(i)] = dmx.add_fixture(Dimmer)
lights['strobe_intensity'] = dmx.add_fixture(Dimmer)
lights['strobe_speed'] = dmx.add_fixture(Dimmer)

/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 5.656331380208332ms, took 22.322998046875ms
  warn("Ticker loop behind by {:,}ms, took {:,}ms".format(-wait_dur, loop_dur))


In [122]:
# turn 1-4 on
lights['1'].dim(255)
lights['2'].dim(255)
lights['3'].dim(255)
lights['4'].dim(255)

In [123]:
# turn 1-4 off
lights['1'].dim(0)
lights['2'].dim(0)
lights['3'].dim(0)
lights['4'].dim(0)

/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 3.478108723958332ms, took 20.144775390625ms
  warn("Ticker loop behind by {:,}ms, took {:,}ms".format(-wait_dur, loop_dur))


In [124]:
# turn on strobe full blast for one second
lights['strobe_intensity'].dim(255)
lights['strobe_speed'].dim(255)
time.sleep(1)
lights['strobe_intensity'].dim(0)
lights['strobe_speed'].dim(0)


/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 3.295247395833332ms, took 19.9619140625ms
  warn("Ticker loop behind by {:,}ms, took {:,}ms".format(-wait_dur, loop_dur))
/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 2.326497395833332ms, took 18.9931640625ms
  warn("Ticker loop behind by {:,}ms, took {:,}ms".format(-wait_dur, loop_dur))
/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 3.425374348958332ms, took 20.092041015625ms
  warn("Ticker loop behind by {:,}ms, took {:,}ms".format(-wait_dur, loop_dur))
/Users/f004swn/anaconda3/envs/spotify/lib/python3.12/site-packages/PyDMXControl/utils/timing/_Ticker.py:79: UserWarning: Ticker loop behind by 3.807454427083332ms, took 20.47412109375ms
  warn("Ticker lo

In [111]:
# set max strobe intensity and speed
time.sleep(1)
lights['strobe_intensity'].on()
lights['strobe_speed'].on()
time.sleep(5)
lights['strobe_intensity'].dim(0, 0)
lights['strobe_speed'].dim(0, 0)

In [27]:

# Add a new Dimmer fixture to our controller
#  and save it to a variable so we can access it.
# We give it a name so it's easier to identify in the debug control options.
fixture = dmx.add_fixture(Dimmer, name="My_First_Dimmer")


In [28]:

# Next, dim the intensity of the fixture from it's initial value of zero
#  to full, which is represented as 255 in DMX.
# This is done over 5000 milliseconds, or 5 seconds.
fixture.dim(255, 0)


In [24]:
fixture.dim(255, 0)

In [18]:
# slowly dim the intensity of the fixture from full to zero
#  over 10 seconds.
fixture.dim(0, 1000)
time.sleep(1)
fixture.dim(255, 1000)
time.sleep(1)
fixture.dim(0, 1000)
time.sleep(1/10)
fixture.dim(255, 1000)

In [19]:
# set the strobe frequency in hz
hz = 10 # 10Hz, 10 flashes per second
while True:
    time.sleep(1/hz)
    fixture.dim(255)
    fixture.dim(0, 0)

KeyboardInterrupt: 